In [1]:
import pandas as pd
import numpy as np
from datetime  import datetime  
from datetime import timedelta  
import lightgbm as lgb
from sklearn import preprocessing, metrics
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.lightgbm

In [2]:
X_train = pd.read_pickle('../01_preprocessed_data/X_1_train.pkl')
y_train = pd.read_pickle('../01_preprocessed_data/y_1_train.pkl')
X_val = pd.read_pickle('../01_preprocessed_data/X_1_val.pkl')
y_val = pd.read_pickle('../01_preprocessed_data/y_1_val.pkl')
X_test = pd.read_pickle('../01_preprocessed_data/X_1_test.pkl')

In [3]:
data_location = '../00_data/sample_submission.csv'
submission = pd.read_csv(data_location)

In [4]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)

(37915775, 73)
(853632, 73)
(853720, 73)
(37915775, 1)
(853632, 1)


In [17]:
params = {
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'objective':  'poisson',
        'n_jobs': -1,
        'seed': 0,
        'learning_rate': 0.1, 
        'bagging_fraction': 0.75,
        'bagging_freq': 10, 
        'colsample_bytree': 0.75,
        'num_iterations': 1000,
        'early_stopping_round': 100}

In [18]:
data_location = '../04_submissions/lightGBM_no_fe(7)(1).csv'
best_submission_original = pd.read_csv(data_location)
best_submission = pd.read_csv(data_location)

In [19]:
evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]


In [ ]:
features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 
            'weekday', 'month', 'year',
            'event_name_1',  
            'snap_CA', 'snap_TX', 'snap_WI',
            'days_from_start', 'sell_price', 'avg_price', 
            'ADI', 'CV2', 'demand_type',
           'avg_weekday_demand', 'avg_demand', 'max_demand', 'quantile025_week', 'max_weekday_demand',  
            'lag_28', 'lag_35']
for i in range(1, 7):
    print(i)
    features_for_run = features + ["lag_{}".format(i)] + ["lag_{}".format(i+1)] + ['rmean_{}_7'.format(i)] + \
    ['rmean_{}_7'.format(i+1)] + ['rmean_{}_28'.format(i+1)]
    train_set = lgb.Dataset(X_train[features_for_run], y_train)
    val_set = lgb.Dataset(X_val[features_for_run], y_val)
    model = lgb.train(params, train_set, valid_sets = [train_set, val_set], verbose_eval = 100)
    y_test = model.predict(X_test[features_for_run])
    X_test['demand'] = y_test
    predictions = X_test[['id', 'days_from_start', 'demand']]
    predictions = pd.pivot(predictions, index = 'id', columns = 'days_from_start', values = 'demand').reset_index()
    predictions.columns = ['id'] + ['F' + str(j + 1) for j in range(28)]
    validation = submission[['id']].merge(predictions, on = 'id')

    final = pd.concat([validation, evaluation])
    final.to_csv('../04_submissions/final-{}.csv'.format(i), index = False)
    best_submission['F{}'.format(i)] = final['F{}'.format(i)]

1
Training until validation scores don't improve for 100 rounds


In [ ]:
best_submission.to_csv('../04_submissions/best_submission_day1-{}.csv'.format(i), index = False)

In [ ]:
with open("../04_submissions/best_submission_day1-{}.txt".format(i), "a") as outfile:
    outfile.write("\n".join(features))
    outfile.write("\n")
    outfile.write(str(params) )
    outfile.write("\n")
    #outfile.write(str(val_score))
    #outfile.write("\n")

In [ ]:
best_submission

In [ ]:
best_submission_original